# Initial Setup

In [1]:
import os
import sys
import json
import string
import pathlib
import logging
import pandas as pd
import gradio as gr
from typing import Literal
from dotenv import load_dotenv
from openai import OpenAI
from datetime import datetime, timedelta

logger = logging.getLogger()
fhandler = logging.FileHandler(filename="chatbot_tools.log", mode="a")
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.DEBUG)

## LLMs

In [2]:
# Local Bot
OLLAMA_API_KEY = "ollama"
OLLAMA_BASE_URL = "http://localhost:11434/v1"
llama_client = OpenAI(api_key=OLLAMA_API_KEY, base_url=OLLAMA_BASE_URL)
llama_model = "llama3.2"

# Online Bot
load_dotenv(override=True)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "type-your-api-key-here")
gpt_client = OpenAI()
gpt_model = "gpt-4o-mini"

## Data

In [3]:
ABSOLUTE_PATH = os.path.abspath(os.getcwd())
data_dir = "data"

path_room_type = os.path.join(ABSOLUTE_PATH, data_dir, "hotel_room_types.csv")
path_booking_data = os.path.join(ABSOLUTE_PATH, data_dir, "hotel_booking_data.csv")
path_room_availability = os.path.join(ABSOLUTE_PATH, data_dir, "room_availability.csv")

## Helper Function

### Dataset Related Functions

In [4]:
def load_dataset(file_path: str) -> pd.DataFrame:
    return pd.read_csv(file_path)

def save_dataset(df: pd.DataFrame, file_path: str):
    df.to_csv(file_path, index=False)

def convert_to_records(df: pd.DataFrame) -> list:
    records = df.to_json(orient="records")
    return json.loads(records)

### Booking Related Functions

In [5]:
def generate_booking_id() -> str:
    booking_data = load_dataset(path_booking_data)
    last_booking_id = list((booking_data.Booking_ID))[-1]
    last_booking_id = int(last_booking_id[1:])
    new_booking_id = last_booking_id + 1
    return "B" + f"{new_booking_id:03d}" 

def add_booking_data(new_booking_data: dict):
    booking_data = load_dataset(path_booking_data)
    booking_data.loc[len(booking_data)] = new_booking_data
    
    save_dataset(booking_data, path_booking_data)

### Room Related Functions

In [6]:
def get_room_type() -> list:
    room_type = load_dataset(path_room_type)
    return list(room_type.Room_Type)

def get_max_occupancy(room_type: str) -> int:
    room_types = load_dataset(path_room_type)
    max_occupancy = room_types.Max_Occupancy[room_types.Room_Type == room_type].iloc[0]
    return int(max_occupancy)

def get_room_availability(date: str) -> list:
    all_rooms = load_dataset(path_room_availability)
    rooms = all_rooms[all_rooms.Date == date].iloc[:, 1:]
    rooms = convert_to_records(rooms)
    
    if not rooms:
        room_type = get_room_type()
        for room in room_type:
            rooms.append({"Room_Type": room, "Available_Rooms": 5})
    return rooms

def update_room_availability(room_type: str, checkin_date: str, nights: int, update_type: Literal["booking", "cancellation"]):
    df = load_dataset(path_room_availability)
    day = datetime.fromisoformat(checkin_date)
    for night in range(0, nights):
        date = day.strftime('%Y-%m-%d')

        if update_type == "booking":
            df.loc[(df.Date == date) & (df.Room_Type == room_type), "Available_Rooms"] -= 1
        elif update_type == "cancellation":
            df.loc[(df.Date == date) & (df.Room_Type == room_type), "Available_Rooms"] += 1
        
        day = day + timedelta(days=1)
    
    save_dataset(df, path_room_availability)

### Calculation related functions

In [7]:
def calculate_nights(start_date: str, end_date: str) -> int:
    start = datetime.fromisoformat(start_date)
    end = datetime.fromisoformat(end_date)
    return (end - start).days + 1

def calculate_price(room_type: str, nights: int) -> float:
    room_types = load_dataset(path_room_type)
    price_per_night = room_types.Price_Per_Night[room_types.Room_Type == room_type].iloc[0]
    price_per_night = float(price_per_night[1:])
    return price_per_night * nights

# Tools

## get_room_details

In [8]:
def get_room_details(room_type: str) -> dict:
    room_type = room_type.title()
    
    rooms = load_dataset(path_room_type)
    room_details = rooms[rooms.Room_Type == room_type]

    if not room_details.empty:
        room_details = convert_to_records(room_details)[0]
    else:
        room_details = {"Room_Type": "Unknown", "Room Details": "Unknown"}
    
    return room_details

# Testing
get_room_details("suite room")

{'Room_Type': 'Suite Room',
 'Description': 'A luxurious suite with a separate living area and premium amenities',
 'Facility': 'Free Wi-Fi, King Bed, Bathtub, Lounge Area, Coffee Maker',
 'Price_Per_Night': '$200',
 'Max_Occupancy': 2}

In [9]:
get_room_details_function = {
    "name": "get_room_details",
    "description": "Get room details, including description, facilities, price per night, and maximum occupancy based on room type. Call this whenever you need to know the details of the room type, for example when a customer asks 'Tell me more about this room type'",
    "parameters": {
        "type": "object",
        "properties": {
            "room_type": {
                "type": "string",
                "description": "The room type that customers want to know about"
            }
        },
        "required": ["room_type"],
        "additionalProperties": False
    }
}

## check_room_type_availability

In [10]:
def check_room_type_availability(room_type: str, start_date: str, end_date: str) -> list:
    room_type = room_type.title()
    
    # Check room type
    if not room_type in get_room_type():
        return [{"Room Type": "Unknown", "Availability_Status": "Unknown"}]
    
    day = datetime.fromisoformat(start_date)
    nights = calculate_nights(start_date, end_date)
    if nights <= 0:
        return [{"Date": "Error", "Availability_Status": "Unknown"}]
    
    report = []
    for night in range(0, nights):
        date = day.strftime('%Y-%m-%d')
        room_availability = get_room_availability(date)
        room_availability = {item['Room_Type']: item['Available_Rooms'] for item in room_availability}
        room_type_availability = room_availability[room_type]

        report.append({
            "Date": date,
            "Availability_Status": f"{room_type_availability} rooms available" if room_type_availability else "fully booked"
        })
        day = day + timedelta(days=1)
    return report

# Testing
check_room_type_availability("single room", "2025-05-17", "2025-05-19")

[{'Date': '2025-05-17', 'Availability_Status': '5 rooms available'},
 {'Date': '2025-05-18', 'Availability_Status': '3 rooms available'},
 {'Date': '2025-05-19', 'Availability_Status': 'fully booked'}]

In [11]:
check_room_type_availability_function = {
    "name": "check_room_type_availability",
    "description": "Get room type availability data. Call this whenever you need to know the availability of a particular room type during a specific time period, for example when a customer asks 'Is this room type available from this date to this date'",
    "parameters": {
        "type": "object",
        "properties": {
            "room_type": {
                "type": "string",
                "description": "The type of room the customer wants to check the availability of"
            },
            "start_date": {
                "type": "string",
                "description": "Room availability check start date. This date must be in YYYY-MM-DD format. Therefore, change the date format to YYYY-MM-DD first."
            },
            "end_date": {
                "type": "string",
                "description": "Room availability check end date. This date must be in YYYY-MM-DD format. Therefore, change the date format to YYYY-MM-DD first."
            }
        },
        "required": ["room_type", "start_date", "end_date"],
        "additionalProperties": False
    }
}

## book_a_room

In [12]:
def book_a_room(customer_name: str, room_type: str, checkin_date: str, checkout_date: str, num_guest: int) -> str:
    room_type = room_type.title()
    
    # Check room type
    if not room_type in get_room_type():
        return "Failed to process your booking. Room type unknown."

    # Check date
    nights = calculate_nights(checkin_date, checkout_date)
    if nights <= 0:
        return "Failed to process your booking. Check out date is earlier than check in date."

    # Check room availability
    report = check_room_type_availability(room_type, checkin_date, checkout_date)
    fully_booked = [row for row in report if row["Availability_Status"] == "fully booked"]
    fully_booked_dates = ", ".join([row["Date"] for row in fully_booked])
    if fully_booked:
        return f"Failed to process your booking. {room_type} are fully booked on these dates {fully_booked_dates}."
    
    # Add booking data
    booking_id = generate_booking_id()
    total_price = "$" + str(calculate_price(room_type, nights))
    booking_status = "Booked"
    
    new_booking_data = {
        "Booking_ID": booking_id,
        "Customer_Name": customer_name,
        "Room_Type": room_type,
        "Check_In_Date": checkin_date,
        "Check_Out_Date": checkout_date,
        "Num_Guests": num_guest,
        "Total_Price": total_price,
        "Booking_Status": booking_status
    }
    add_booking_data(new_booking_data)

    # Update room availability
    update_room_availability(room_type, checkin_date, nights, update_type="booking")
    
    status = f"Your booking is successful with the following details:\n"
    status += f"{new_booking_data}\n"
    status += f"Please save and remember your booking ID: {booking_id}\n"
    if num_guest > get_max_occupancy(room_type):
        status += "The number of guests exceeds the maximum occupancy, there will be an additional charge."
        
    return status

In [13]:
# Test
test = book_a_room(
    customer_name="Yusup",
    room_type="suite room",
    checkin_date="2025-05-18",
    checkout_date="2025-05-18",
    num_guest=5
)
print(test)

Your booking is successful with the following details:
{'Booking_ID': 'B010', 'Customer_Name': 'Firlyana', 'Room_Type': 'Suite Room', 'Check_In_Date': '2025-05-18', 'Check_Out_Date': '2025-05-18', 'Num_Guests': 5, 'Total_Price': '$200.0', 'Booking_Status': 'Booked'}
Please save and remember your booking ID: B010
The number of guests exceeds the maximum occupancy, there will be an additional charge.


In [14]:
book_a_room_function = {
    "name": "book_a_room",
    "description": "To book a room. Call this whenever you need to book a specific room type for a specific period of time for a customer, for example when a customer asks 'I would like to book a room at your hotel'",
    "parameters": {
        "type": "object",
        "properties": {
            "customer_name" : {
                "type": "string",
                "description": "Name of the customer who booked the room"
            },
            "room_type": {
                "type": "string",
                "description": "The type of room the customer booked"
            },
            "checkin_date": {
                "type": "string",
                "description": "Check-in date. This date must be in YYYY-MM-DD format. Therefore, change the date format to YYYY-MM-DD first."
            },
            "checkout_date": {
                "type": "string",
                "description": "Check-out date. This date must be in YYYY-MM-DD format. Therefore, change the date format to YYYY-MM-DD first."
            },
            "num_guest": {
                "type": "integer",
                "description": "Number of guests who will stay in the room"
            }
        },
        "required": ["customer_name", "room_type", "checkin_date", "checkout_date", "num_guest"],
        "additionalProperties": False
    }
}

## get_booking_data

In [15]:
def get_booking_data(booking_id: str) -> dict:
    all_booking_data = load_dataset(path_booking_data)
    booking_data = all_booking_data[all_booking_data.Booking_ID == booking_id]

    if not booking_data.empty:
        booking_data = convert_to_records(booking_data)[0]
    else:
        booking_data = {"Booking_ID":"Unknown", "Booking_Data": "Unknown"}
        
    return booking_data

# Testing
get_booking_data("B006")

{'Booking_ID': 'B006',
 'Customer_Name': 'Firlyana',
 'Room_Type': 'Suite Room',
 'Check_In_Date': '2025-05-18',
 'Check_Out_Date': '2025-05-18',
 'Num_Guests': 5,
 'Total_Price': '$200.0',
 'Booking_Status': 'Cancelled'}

In [16]:
get_booking_data_function = {
    "name": "get_booking_data",
    "description": "Get room booking details. Call this whenever you need to know the details of a room booking made by a customer based on their booking ID, for example when a customer asks 'I want to see my booking details'",
    "parameters": {
        "type": "object",
        "properties": {
            "booking_id": {
                "type": "string",
                "description": "Customer booking ID"
            }
        },
        "required": ["booking_id"],
        "additionalProperties": False
    }
}

## cancel_booking

In [17]:
def cancel_booking(booking_id: str) -> str:
    # Check booking id
    booking_data = get_booking_data(booking_id)
    if booking_data["Booking_ID"] == "Unknown":
        return f"Failed to process your booking cancellation. Your booking ID was not found."
        
    # Check booking status
    if booking_data["Booking_Status"] == "Cancelled":
        return f"Your booking cancellation has been processed."
    elif booking_data["Booking_Status"] == "Confirmed" or booking_data["Booking_Status"] == "Checked-in":
        return f"You have completed the payment. Contact admin to cancel your room."
    elif booking_data["Booking_Status"] == "Checked-out":
        return f"You have checked out. Thank you for visiting our hotel."
    
    # Get booking details
    booking_id = booking_data["Booking_ID"]
    customer_name = booking_data["Customer_Name"]
    room_type = booking_data["Room_Type"]
    checkin_date = booking_data["Check_In_Date"]
    checkout_date = booking_data["Check_Out_Date"]
    nights = calculate_nights(checkin_date, checkout_date)
    
    # Update booking data
    df = load_dataset(path_booking_data)
    df.loc[(df.Booking_ID == booking_id) & (df.Customer_Name == customer_name), "Booking_Status"] = "Cancelled"
    df.to_csv(path_booking_data, index=False)
    
    # Update room avaiability
    update_room_availability(room_type, checkin_date, nights, update_type="cancellation")

    return f"Your booking cancellation is successful. You can check the cancellation using your Booking ID."

In [18]:
# Test
cancel_booking("B006")

'Your booking cancellation has been processed.'

In [19]:
cancel_booking_function = {
    "name": "cancel_booking",
    "description": "Cancel room reservation. Call this whenever you need to cancel a customer's room reservation based on their booking ID, for example when a customer asks 'I want to cancel my room reservation'",
    "parameters": {
        "type": "object",
        "properties": {
            "booking_id": {
                "type": "string",
                "description": "Customer booking ID"
            }
        },
        "required": ["booking_id"],
        "additionalProperties": False
    }
}

# Prompting

In [20]:
tools = [
    {"type": "function", "function": get_room_details_function},
    {"type": "function", "function": check_room_type_availability_function},
    {"type": "function", "function": book_a_room_function},
    {"type": "function", "function": get_booking_data_function},
    {"type": "function", "function": cancel_booking_function}
]
tools

[{'type': 'function',
  'function': {'name': 'get_room_details',
   'description': "Get room details, including description, facilities, price per night, and maximum occupancy based on room type. Call this whenever you need to know the details of the room type, for example when a customer asks 'Tell me more about this room type'",
   'parameters': {'type': 'object',
    'properties': {'room_type': {'type': 'string',
      'description': 'The room type that customers want to know about'}},
    'required': ['room_type'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'check_room_type_availability',
   'description': "Get room type availability data. Call this whenever you need to know the availability of a particular room type during a specific time period, for example when a customer asks 'Is this room type available from this date to this date'",
   'parameters': {'type': 'object',
    'properties': {'room_type': {'type': 'string',
      'description

In [21]:
def handle_tool_call(message):
    responses = []

    for tool_call in message.tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        
        logging.debug(f"Calling {tool_name}.")
        logging.debug(f"Arguments: {arguments}")
        
        if tool_name == "get_room_details":
            room_type = arguments.get("room_type")
            room_details = get_room_details(room_type)
            content = {"room_type": room_type, "room_details": room_details}
            
        elif tool_name == "check_room_type_availability":
            room_type = arguments.get("room_type")
            start_date = arguments.get("start_date")
            end_date = arguments.get("end_date")
            report = check_room_type_availability(room_type, start_date, end_date)
            content = {
                "room_type": room_type,
                "start_date": start_date,
                "end_date": end_date,
                "report": report
            }
            
        elif tool_name == "book_a_room":
            customer_name = arguments.get("customer_name")
            room_type = arguments.get("room_type")
            checkin_date = arguments.get("checkin_date")
            checkout_date = arguments.get("checkout_date")
            num_guest = arguments.get("num_guest")
            status = book_a_room(customer_name, room_type, checkin_date, checkout_date, num_guest)
            content = {
                "customer_name": customer_name,
                "room_type": room_type,
                "checkin_date": checkin_date,
                "checkout_date": checkout_date,
                "num_guest": num_guest,
                "status": status
            }
            
        elif tool_name == "get_booking_data":
            booking_id = arguments.get("booking_id")
            booking_data = get_booking_data(booking_id)
            content = {"booking_id": booking_id, "booking_data": booking_data}
            
        elif tool_name == "cancel_booking":
            booking_id = arguments.get("booking_id")
            status = cancel_booking(booking_id)
            content = {"booking_id": booking_id, "status": status}
            
        else:
            content = {"error": f"Unknown tool: {tool_name}"}

        responses.append({
            "role": "tool",
            "content": json.dumps(content),
            "tool_call_id": tool_call.id
        })

    return responses

In [22]:
system_message = "You are an assistant who helps for the Kabil Hotel called kAIbil. "
system_message += f"Kabil Hotel offers 4 types of rooms including: {', '.join(get_room_type())}. "
system_message += "Respond to customers enthusiastically, politely, and professionally. "
system_message += "Always be accurate. If you don't know the answer, say so and encourage the customer to look at the information on the website or contact customer service. "
system_message += "The hotel website can be accessed here www.kabilhotels.com. "
system_message += "Customer service contact number is 711870. "
system_message += "The hotel location can be accessed here https://g.co/kgs/Wqk4SVK."

def chat(message, history, client: OpenAI=llama_client, model: str=llama_model):
    """Gradio dedicated function for chatbot"""
    
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    response = client.chat.completions.create(
        model=model,
        messages=messages,
        tools=tools,
        temperature=0.3
    )
    
    logging.debug(f"Finish reason: {response.choices[0].finish_reason}")
    if response.choices[0].finish_reason == "tool_calls":
        tool_message = response.choices[0].message
        tool_calls = tool_message.tool_calls
        
        tool_responses = handle_tool_call(tool_message)

        messages.append(tool_message)
        messages.extend(tool_responses)

        response = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=0.3
        )
        
    return response.choices[0].message.content

# UI

In [26]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.
